In [1]:
! pip install langchain_core langchain-anthropic langgraph 

  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Initialize an LLM

In [ ]:
import os
import getpass
#from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic

#load_dotenv()

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Load ANTHROPIC_API_KEY from .env
#key = os.getenv("ANTHROPIC_API_KEY")
#print(key)

_set_env("ANTHROPIC_API_KEY")

llm = ChatAnthropic(model="claude-3-5-sonnet-latest")

## Augmented LLM
![Augmented LLM](images/augmented-llm.png)

In [6]:
# Schema for structured output
from pydantic import BaseModel, Field
class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query that is optimized web search.")
    justification: str = Field(
        None, justification="Why this query is relevant to the user's request."
    )

# Augment the LLM with schema for structured output
structured_llm = llm.with_structured_output(SearchQuery)

# Invoke the augmented LLM
output = structured_llm.invoke("How does Calcium CT score relate to high cholesterol?")
print(output)
print(output.search_query)
print(output.justification)

search_query='relationship between calcium CT score coronary calcium and high cholesterol levels' justification='This search will help find scientific evidence linking coronary calcium scores from CT scans to cholesterol levels and cardiovascular risk.'
relationship between calcium CT score coronary calcium and high cholesterol levels
This search will help find scientific evidence linking coronary calcium scores from CT scans to cholesterol levels and cardiovascular risk.


In [7]:
# Define a tool
def multiply(a: int, b: int) -> int:
    return a * b

# Augment the LLM with tools
llm_with_tools = llm.bind_tools([multiply])

# Invoke the LLM with input that triggers the tool call
msg = llm_with_tools.invoke("What is 2 times 3?")

# Get the tool call
msg.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'toolu_01JadrTooVwrYVDHBpgRKNKz',
  'type': 'tool_call'}]

## Prompt Chaining
![Prompt Chaining](images//prompt-chaining.png)


In [9]:
from typing_extensions import TypedDict

# Graph state
class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str